In [193]:
import sys
import json

params = {'period':'am'}
default = {'training_folder': '../..', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../..', 'params': {'period': 'am'}}


In [195]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel


In [196]:
period = argv['params']['period']
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)
training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if on_lambda:
    scenario_folder = training_folder +r'/inputs/'
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
else:
    scenario_folder = training_folder + '/scenarios/' + period + '/inputs/'
    output_folder = training_folder + '/scenarios/' + period + '/outputs/'
    model_folder = training_folder + '/scenarios/' + period + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)


pt_folder = scenario_folder + 'pt/'
road_folder = scenario_folder + 'road/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/am/outputs/
scen folder :  ../../scenarios/am/inputs/
model folder :  ../../scenarios/am/model/


In [190]:
var, ancestry = excel.read_var(file=input_folder+'/parameters.xlsx', scenario=period, return_ancestry=True)


# inputs

In [202]:
links = gpd.read_file(pt_folder + 'links.geojson') 
nodes = gpd.read_file(pt_folder + 'nodes.geojson')
links = links.set_index('index')
nodes = nodes.set_index('index')

In [203]:
rnodes = gpd.read_file(road_folder + 'road_nodes.geojson')
rnodes = rnodes.set_index('index')
rlinks = gpd.read_file(road_folder + 'road_links.geojson')
rlinks = rlinks.set_index('index')

In [204]:
filepath = excel.get_filepath(input_folder + '{s}/zones/zones.geojson', ancestry=ancestry, log=False)
zones = gpd.read_file(filepath)
zones = zones.set_index('index')

# create Model

In [206]:
sm = stepmodel.StepModel(epsg=4326)
sm.links = links
sm.nodes = nodes
sm.road_links = rlinks
sm.road_nodes = rnodes
sm.zones = zones

In [207]:
centroid = [*LineString(sm.nodes.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])
crs

32618

In [208]:
sm = sm.change_epsg(crs,'meter')

Reprojecting model from epsg 4326 to epsg 32618: 100%|█| 11/11 [00:05<00:00,  2.


# clustering and walkmodel

In [209]:
sm.preparation_clusterize_nodes(distance_threshold=100)


In [210]:
sm.preparation_footpaths(max_length = var['footpaths']['max_length'], speed = var['footpaths']['speed'])


In [211]:
# Zone to transit
sm.preparation_ntlegs(
    short_leg_speed = var['zone_to_transit']['speed'],
    long_leg_speed = var['zone_to_transit']['speed'], # tout le monde marche
    threshold = 1000,
    zone_to_transit = True,
    zone_to_road = False,
    road_to_transit = False,
    n_ntlegs = var['zone_to_transit']['nbr'],
    max_ntleg_length = var['zone_to_transit']['max_length'],
)
sm._add_type_prefixes({'zone_to_transit':'ztt_'})

In [13]:
with open(output_folder + 'test.txt', 'w') as f:
    f.write('test passed! \n')
    f.write('argv: ')
    f.write(str(argv))